# Generating 100,000 FSPS Spectra with Diverse SFHs

Goal is to get a more thoroughly sampled manifold, with more diverse star formation histories spanning a wider range of mean ages...

Updated 2017-02-07 to use zcontinuous=1 (vs. integer values for metallicity)

Updated 2017-03-14 to add nebular emission (gas_logu fixed to default and gas_logz equal to logzsol) and allow more dust properties to vary (dust1 fixed to 2xdust2, and dust_index set equal to dust1_index)

In [ ]:
# imports and plotting setup
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import fsps
from astroML.plotting import setup_text_plots
import h5py

np.set_printoptions(suppress=True)
setup_text_plots(fontsize=24)
mpl.rc('xtick', labelsize=18)
mpl.rc('ytick', labelsize=18)
mpl.rc('font', size=24, family='serif', style='normal', variant='normal', stretch='normal', weight='bold')
mpl.rc('legend', labelspacing=0.1, handlelength=2, fontsize=13)
mpl.rc('axes', labelweight='black')

#### Generate random values of all parameters. Need to think more about what are appropriate distributions for each of these parameters -- just using uniform (either linear or log) sampling in each parameter for now...

In [ ]:
# get 100,000 samples
size=100000

#sf_begin = np.random.uniform(low=0.0, high=12.2, size=size)
logsf_begins = np.random.uniform(low=np.log10(0.1), high=np.log10(12.2), size=size)
sf_begins = 10**logsf_begins
As = 13.7 - sf_begins

logtbursts = np.array([])
for ii in range(size):
    logtbursts = np.append(logtbursts, np.random.uniform(low=np.log10(sf_begins[ii]), high=np.log10(13.6)))
tbursts = 10**logtbursts

logfbursts = np.random.uniform(low=np.log10(0.01), high=np.log10(0.5), size=size)
fbursts = 10**logfbursts

logtaus = np.random.uniform(low=-0.5, high=1.5, size=size)
taus = 10**logtaus
tau_ages = As - taus * ( (1. - (As/taus + 1.) * np.exp(-As/taus)) / (1. - np.exp(-As/taus)) ) # mean ages given taus

total_ages = (1-fbursts) * tau_ages + fbursts * (13.7 - tbursts)

dusts = np.random.uniform(low=0.0, high=1.0, size=size)
dust_slopes = np.random.uniform(low=-1.3, high=-0.3, size=size)

zs = np.random.uniform(low=-1.5, high=0.2, size=size) #not an ideal distribution; will get a lot of low metallicities...

#sigmas = np.random.uniform(low=0.0, high=300.0, size=size)

In [ ]:
n, bins, patches = plt.hist(total_ages, bins=50)

In [ ]:
np.max(sf_begins)

#### now build the model spectra

In [ ]:
spectra = np.zeros((size, 4220)) #know there are 4220 steps in desired wavelength range
mags = np.zeros((size, 5)) # save mags in 5 SDSS filters
bands = fsps.find_filter('sdss') # u, g, i, r, z (NOTE SILLY ORDERING)
ii = 0

for sf_begin, tburst, fburst, tau, dust, dust_slope, z  in zip(sf_begins, tbursts, fbursts, taus, dusts, dust_slopes, zs):
    sp = fsps.StellarPopulation(compute_vega_mags=False, sfh=4, sigma_smooth=100., dust_type=2,
                                add_neb_emission=True, add_neb_continuum=True, sf_start=sf_begin, 
                                tburst=tburst, fburst=fburst, tau=tau, dust1=2*dust, dust2=dust,
                                dust_index=dust_slope, dust1_index=dust_slope, zcontinuous=1,
                                logzsol=z, gas_logz=z, gas_logu=-2.0)
    wave, spec = sp.get_spectrum(tage=13.7, peraa=True)
    wh = (wave < 7400.) * (wave > 3600.)
    spectra[ii, :] = spec[wh]
    mags[ii, :] = sp.get_mags(bands=fsps.find_filter('sdss'), tage=13.7)
    ii += 1
    if ii%500 == 0:
        percent = ii/1000.
        print "%g percent complete" % percent

#### normalize the spectra

In [ ]:
norms = spectra.shape[1] / np.sum(spectra, axis=1)
spectra_norm = spectra * norms[:,np.newaxis]

In [ ]:
# quick sanity check
plt.figure()
plt.plot(wave[wh], spectra_norm[100])
plt.plot(wave[wh], spectra_norm[1009])
plt.plot(wave[wh], spectra_norm[6095])
plt.plot(wave[wh], spectra_norm[9998])
plt.plot(wave[wh], spectra_norm[50628])
plt.plot(wave[wh], spectra_norm[82534])

#### save spectra and parameters to HDF5 file

In [ ]:
f = h5py.File('../data/1e5_spectra_diverseSFH_elines_17-03-14.hdf5','w')
f.create_dataset('spectra', data=spectra_norm)
f.create_dataset('wave', data=wave[wh])
f.create_dataset('sdss_mags', data=mags)
f.create_dataset('sf_begins', data=sf_begins)
f.create_dataset('tbursts', data=tbursts)
f.create_dataset('fbursts', data=fbursts)
f.create_dataset('taus', data=taus)
f.create_dataset('mean_ages', data=total_ages)
f.create_dataset('zs', data=zs)
f.create_dataset('dusts', data=dusts)
f.create_dataset('dust_slopes', data=dust_slopes)

In [ ]:
f.keys()

In [ ]:
f['spectra'].shape

In [ ]:
f.close()